In [1]:
from bert_serving.client import BertClient
from keras.models import model_from_json
import pickle
import json
from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
model_name = 'model_0.7975'

In [3]:
def split_lyrics(coll, num):
    res = []
    splitted = coll.split()
    start = 0
    end = start + num
    length = len(splitted)
    while start < length:
        res.append(" ".join(splitted[start:end]))
        start = end
        end += num
    return res

In [4]:
def normalize_lyrics(text):
    res = text.replace('\n', '. ')
    res = res.replace('..', '.')
    res = res.replace('..', '.')
    res = split_lyrics(res, 25)
    return res
    

In [5]:
def encode(normalized): 
    bc = BertClient()
    return bc.encode(normalized)

In [6]:
def classify(encoded, model_name):
    json_file = open(model_name + ".json", "r")
    loaded_model_json = json_file.read()
    json_file.close()

    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(model_name + ".h5")
    
    #compile_model
    loaded_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    res = loaded_model.predict(encoded)
    
    return res

In [7]:
# def classify_lyrics(id, lyrics, model_name):
#     normalized = normalize_lyrics(lyrics)
#     encoded = encode(normalized)
    
#     with open('classify_lyric.bin', 'wb') as out:
#         pickle.dump(encoded, out)
# #     res = classify(encoded, model_name)
# #     final_res = make_final(res)
# #     return final_res

In [8]:
# classify_lyrics("There's things that you guess\nAnd things that you know\nThere's boys that you can trust\nAnd girls that you don't\n\nThere's little things you hide\nAnd little things that you show\nSometimes you think you're gonna get it\nBut you don't and that's just the way it goes\n\nI swear I won't tease you\nWon't tell you no lies\nI don't need no bible\nJust look in my eyes\n\nI've waited so long baby\nNow that we're friends\nEvery man's got his patience\nAnd here's where mine ends\n\nI want your sex\nI want your love\nI want your sex\nI want your... sex\n\nIt's playing on my mind\nIt's dancing on my soul\nIt's taken so much time\nSo why don't you just let me go\n\nI'd really like to try\nOh, I'd really love to know\nWhen you tell me you're gonna regret it\nThen I tell you that I love you but you still say NO\n\nI swear I won't tease you\nWon't tell you no lies\nI don't need no bible\nJust look in my eyes\n\nI've waited so long baby\nOut in the cold\nI can't take much more girl\nI'm losing control\n\nI want your sex\nI want your love\nI want your sex\nI want your... sex\n\nIt's natural\nIt's chemical (Let's do it)\nIt's logical\nHabitual (Can we do it?)\n\nIt's sensual\nBut most of all...\nSex is something that we should do\nSex is something for me and you\n\nSex is natural, sex is good\nNot everybody does it\nBut everybody should\nSex is natural, sex is fun\nSex is best when it's... one on one\nOne on one\n\nI'm not your father\nI'm not your brother\nTalk to your sister\nI am a lover\n\nC-c-c-c-come on\n\nWhat's your definition of dirty baby\nWhat do you consider pornography\nDon't you know I love it till it hurts me baby\nDon't you think it's time you had sex with me\n\nWhat's your definition of dirty baby\nWhat do you call pornography\nDon't you know I love it till it hurts me baby\nDon't you think it's time you had sex with me\n\nSex with me\nSex with me\nHave sex with me\n\nC-c-c-c-come on",
#                "models\\" + model_name)

In [9]:
start = 2
end = 2
names = []
for x in range(start, end + 1):
    names.append(str(x) + "_10000_batch.json")

In [ ]:
for x in names:
    with open("crawler/" + x, 'r') as file:
        loaded = json.load(file)
    encoded = []
    cnt = 1
    for y in loaded[470:]:
        lyrics = normalize_lyrics(y['lyrics'])
        if lyrics == []:
            cnt+=1
            continue;
        encoded_lyrics = encode(lyrics)
        encoded.append({
            "id": y['id'],
            "author": y['author'],
            "encoded": encoded_lyrics,
            "lyrics": y['lyrics'],
            "song": y['song']
        })
        clear_output()
        print(cnt)
        cnt+=1
    new_name = x.replace(".json", ".bin")
    with open("encoded/" + new_name, 'wb') as file:
        pickle.dump(encoded, file)

8937
